In [1]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=ad9084fcacf61b1811717e6f6e2590f89d6317273604e63864c9e3caf687d296
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [2]:
import wikipediaapi
import json
from collections import deque, defaultdict
from tqdm import tqdm
import os

wiki = wikipediaapi.Wikipedia(user_agent="NobelGraphBot/1.0", language="vi")

SEEDS = [
    "Danh sách người đoạt giải Nobel",
    "Danh sách người đoạt giải Nobel Vật lý",
    "Danh sách người đoạt giải Nobel Hóa học",
    "Danh sách người đoạt giải Nobel Sinh lý học và Y học",
    "Danh sách người đoạt giải Nobel Văn học",
    "Danh sách người đoạt giải Nobel Hòa bình",
    "Danh sách người đoạt giải Nobel Kinh tế học",
    "giải Nobel"
]

PERSON_KEYWORDS = [
    "giải Nobel", "người đoạt", "nhà khoa học", "nhà văn",
    "nhà kinh tế", "nhà vật lý", "bác sĩ"
]
CATEGORY_KEYWORDS = ["giải Nobel", "nhà khoa học", "nhà văn", "nhà kinh tế", "nhân vật"]

def save_jsonl(fname, data):
    with open(fname, "a", encoding="utf-8") as f:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def match_category(page):
    try:
        cats = page.categories.keys()
        return any(any(kw.lower() in cat.lower() for kw in CATEGORY_KEYWORDS) for cat in cats)
    except:
        return False

def crawl_nobel_graph(max_depth=2, max_links=500, max_nodes=5000):
    queue = deque([(seed, 0) for seed in SEEDS])
    visited = set()
    indegree = defaultdict(int)

    open("nobel_nodes.jsonl", "w").close()
    open("nobel_edges.jsonl", "w").close()

    pbar = tqdm(total=max_nodes, desc="Crawling Nobel")

    while queue and len(visited) < max_nodes:
        title, depth = queue.popleft()
        if title in visited:
            continue
        visited.add(title)

        page = wiki.page(title)
        if not page.exists():
            continue

        node_data = {
            "id": title,
            "title": title,
            "depth": depth,
            "text": page.summary  # chỉ lưu tóm tắt để gọn
        }
        save_jsonl("nobel_nodes.jsonl", node_data)
        pbar.update(1)

        if depth >= max_depth:
            continue

        for link in list(page.links.keys())[:max_links]:
            if any(prefix in link for prefix in ["Wikipedia:", "Thể loại:", "Tập tin:", "Mẫu:"]):
                continue
            edge = {"source": title, "target": link}
            save_jsonl("nobel_edges.jsonl", edge)
            indegree[link] += 1

            lower_link = link.lower()
            if (
                any(kw in lower_link for kw in PERSON_KEYWORDS)
                or indegree[link] >= 2
                or match_category(wiki.page(link))
            ):
                if link not in visited:
                    queue.append((link, depth + 1))

    pbar.close()
    print(f"✅ Done: {len(visited)} nodes saved to nobel_nodes.jsonl")

crawl_nobel_graph()


Crawling Nobel:  73%|███████▎  | 3671/5000 [1:03:43<23:04,  1.04s/it]

✅ Done: 5000 nodes saved to nobel_nodes.jsonl
